In [1]:
#Dependencies

from bs4 import BeautifulSoup as bs
from splinter import Browser
import pandas as pd
import requests
import time

In [2]:
#URLs of Sites to be scraped

news_url = 'https://mars.nasa.gov/news/'
jpl_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
weather_url = 'https://twitter.com/marswxreport?lang=en'
facts_url ='https://space-facts.com/mars/'
usgs_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'

In [3]:
# Retrieve pages with the requests module

news_response = requests.get(news_url)
jpl_response = requests.get(jpl_url)
weather_response = requests.get(weather_url)

In [4]:
# Create BeautifulSoup objects; parse with 'html.parser'

news_soup = bs(news_response.text, 'html.parser')
weather_soup = bs(weather_response.text, 'html.parser')

In [5]:
# https://splinter.readthedocs.io/en/latest/drivers/chrome.html
!which chromedriver

executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)


/usr/local/bin/chromedriver


# News Scraping

In [6]:
#Scrape Latest Title

#Set Up Splinter Browser
browser.visit(news_url)

news_html = browser.html

news_soup = bs(news_html, 'html.parser')

news_title = news_soup.find('div', class_='content_title').text
print(news_title)

#Scrape Teaser Body
news_p = news_soup.find('div', class_='rollover_description_inner').text

print(news_p)

Scientists to Debate Landing Site for Next Mars Rover
Hundreds of scientists and Mars-exploration enthusiasts gather this week to present, discuss and deliberate the future landing site for NASA's next Red Planet. 


# JPL Scraping

In [7]:
browser.visit(jpl_url)

In [8]:
#Click on Featured Image
browser.visit(jpl_url)
time.sleep(2)
browser.click_link_by_partial_text('FULL IMAGE')
time.sleep(2)
browser.click_link_by_partial_text('more info')
time.sleep(2)
browser.click_link_by_partial_text('.jpg')


In [9]:
#Save HTML of the page to variable

html = browser.html

#Use bs to save image url

jpl_soup = bs(html, 'html.parser')
featured_image_url = jpl_soup.find("img")["src"]
featured_image_url
    
print(featured_image_url)

https://photojournal.jpl.nasa.gov/jpeg/PIA01486.jpg


# Twitter Scraping

In [10]:
import re
mars_weather= weather_soup.find(string=re.compile("Sol"))
print(mars_weather) 

Sol 2171 (2018-09-14), high -12C/10F, low -65C/-84F, pressure at 8.79 hPa, daylight 05:43-17:59


# Mars Facts Scraping

In [11]:
tables = pd.read_html(facts_url)

facts_df = tables[0]

facts_df.head()

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.42 x 10^23 kg (10.7% Earth)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.52 AU)"


In [12]:
facts_df.to_html('facts.html')

# Hemisphere Scraping

In [13]:
#Create List of Mars Hemisphers to iterate over and dictionary to save the results
mars_hemis = ["Valles Marineris Hemisphere", "Cerberus Hemisphere", "Schiaparelli Hemisphere", "Syrtis Major Hemisphere"]
hemisphere_image_urls = []

#Set up Splinter URL

browser.visit(usgs_url)

In [14]:
hemisphere_image_urls = []
for i in range (4):
    time.sleep(2)
    images = browser.find_by_tag('h3')
    images[i].click()
    html = browser.html
    soup = bs(html, 'html.parser')
    partial = soup.find("img", class_="wide-image")["src"]
    img_title = soup.find("h2",class_="title").text
    img_url = 'https://astrogeology.usgs.gov'+ partial
    hemi_data = {"title":img_title,"img_url":img_url}
    hemisphere_image_urls.append(hemi_data)
    browser.back()

In [15]:
hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg'}]